In [1]:
%load_ext autoreload

In [ ]:
""" 
Training set for NATO phonetic alphabets.

Each alphabet is repeated 20 times. 16 instances of each alphabet are used for training and 4 instances for validation. 
Test sets are the entire RAINBOW and GRANDFATHER passages.

0: Alfa
1: Bravo
2: Charlie
3: Delta
4: Echo
5: Foxtrot
6: Golf
7: Hotel
8: India
9: Juliette
10: Kilo
11: Lima
12: Mike
13: November
14: Oscar
15: Papa
16: Quebec
17: Romeo
18: Sierra
19: Tango
20: Uniform
21: Victor
22: Whiskey
23: X-ray
24: Yankee
25: Zulu

DATA is given in a numpy array of dimensions (520, 22, 7500) - (26 alphabets each repeated 20 times, 22 channels, 7500 time samples).
Raw data was filtered using 3rd order Butterworth bandpass filter between 80 and 1000 Hertz.

RUN this file before running Part2_NATOAlphabets_Grandfather.ipynb and Part2_NATOAlphabets_Rainbow.ipynb as this files saves the model weights for testing.
"""

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from spdLearning import spdNN
from spdLearning import optimizers 
from spdLearning import trainTest
from spdLearning import spdNet

In [3]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [4]:
dev = "cpu" 
device = torch.device(dev)

In [5]:
numberAlphabets = 26
trialsPerAlphabet = 20
numberTrials = numberAlphabets * trialsPerAlphabet
numberChannels = 22
windowLength = 7500

In [6]:
subjectNumber = 4
subject = "Subject" + str(subjectNumber)

In [7]:
DATA = np.load("Experiment2/" + subject + "/trainSet.npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
DATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)

In [8]:
labelsByAlphabet = np.array([[i] * trialsPerAlphabet for i in range(numberAlphabets)]).reshape(numberTrials)

Indices =  {}
for i in range(numberAlphabets):
    Indices[i] = []
for i in range(len(labelsByAlphabet)):
    Indices[labelsByAlphabet[i]].append(i)

covariancesLabels = np.zeros((numberAlphabets, trialsPerAlphabet, numberChannels, numberChannels))
for i in range(numberAlphabets):
    for j in range(trialsPerAlphabet):
        covariancesLabels[i, j] = 1/windowLength * ((DATA[Indices[i][j]] @ DATA[Indices[i][j]].T))

In [9]:
trainFeatures = np.zeros((numberAlphabets * 16, numberChannels, numberChannels))
trainLabels = np.zeros((numberAlphabets * 16))
count = 0
for i in range(numberAlphabets):
    trainFeatures[count:count + 4] = covariancesLabels[i, :4]
    trainFeatures[count + 4:count + 8] = covariancesLabels[i, 5:9]
    trainFeatures[count + 8:count + 12] = covariancesLabels[i, 10:14]
    trainFeatures[count + 12:count + 16] = covariancesLabels[i, 15:19]
    trainLabels[count:count + 16] = [i] * 16
    count += 16

valFeatures = np.zeros((numberAlphabets * 4, numberChannels, numberChannels))
valLabels = np.zeros((numberAlphabets * 4))
count = 0
for i in range(26):
    valFeatures[count] = covariancesLabels[i, 4]
    valFeatures[count + 1] = covariancesLabels[i, 9]
    valFeatures[count + 2] = covariancesLabels[i, 14]
    valFeatures[count + 3] = covariancesLabels[i, 19]
    valLabels[count:count + 4] = [i] * 4
    count += 4

In [10]:
trainDataset = BaseDataset(trainFeatures, trainLabels)
valDataset = BaseDataset(valFeatures, valLabels)
trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
valDataloader = DataLoader(valDataset, batch_size = 32, shuffle = False)

In [11]:
numberEpochs = 1000

model = spdNet.learnSPDMatrices(numberAlphabets).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)

lossFunction = nn.CrossEntropyLoss()
spdOptimizer = optimizers.MixOptimizer(model.parameters(), lr = 0.05)

7926


In [12]:
maxValue = 0
for epoch in range(numberEpochs):
    trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, spdOptimizer, lossFunction)
    valLoss, valAccuracy = trainTest.testOperation(model, device, valDataloader, lossFunction)
    if maxValue < valAccuracy:
        maxValue = valAccuracy
        torch.save(model.state_dict(), 'Experiment2/' + subject + '/spdNet.pt')
    print(f'Epoch: {epoch + 1}/{numberEpochs}, Training loss: {trainLoss:.4f}, Training accuracy: {trainAccuracy:.2f}%, Val loss: {valLoss:.4f}, Val accuracy: {valAccuracy:.2f}%')
print(maxValue)

Epoch: 1/1000, Training loss: 0.1026, Training accuracy: 3.85%, Val loss: 0.1236, Val accuracy: 4.81%
Epoch: 2/1000, Training loss: 0.1018, Training accuracy: 5.53%, Val loss: 0.1227, Val accuracy: 3.85%
Epoch: 3/1000, Training loss: 0.1008, Training accuracy: 7.93%, Val loss: 0.1225, Val accuracy: 5.77%
Epoch: 4/1000, Training loss: 0.1004, Training accuracy: 9.38%, Val loss: 0.1219, Val accuracy: 13.46%
Epoch: 5/1000, Training loss: 0.0998, Training accuracy: 8.89%, Val loss: 0.1212, Val accuracy: 13.46%
Epoch: 6/1000, Training loss: 0.0992, Training accuracy: 12.50%, Val loss: 0.1207, Val accuracy: 25.96%
Epoch: 7/1000, Training loss: 0.0985, Training accuracy: 14.66%, Val loss: 0.1197, Val accuracy: 29.81%
Epoch: 8/1000, Training loss: 0.0978, Training accuracy: 20.91%, Val loss: 0.1188, Val accuracy: 23.08%
Epoch: 9/1000, Training loss: 0.0972, Training accuracy: 18.75%, Val loss: 0.1181, Val accuracy: 31.73%
Epoch: 10/1000, Training loss: 0.0966, Training accuracy: 22.84%, Val lo

In [13]:
print(maxValue)

89.42307692307692


In [14]:
%autoreload